# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [30]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-10-29	Data-driven Shape Sensing of Continuum Dexterous Manipulators Using Embedded Capacitive Sensor	IEEE Sensor Conference 2023		Q. Li, W. Wang, J. Liu, A. Jain and M. Armand, "Data-driven Shape Sensing of Continuum Dexterous Manipulators Using Embedded Capacitive Sensor," 2023 IEEE SENSORS, Vienna, Austria, 2023, pp. 1-4, doi: 10.1109/SENSORS56945.2023.10324929.	paper-1	https://ieeexplore.ieee.org/abstract/document/10324929
   2024-12-19	Shape Sensing of Continuum Robots with Additively Manufactured Stretchable Strain Sensors: Approach and Preliminary Results	IEEE Robotics and Automation Letters	excerpt	D. C. Moyer, W. Wang, L. S. Karschner, L. Fichera, and P. M. Rao, “Shape sensing of continuum robots with additively manufactured stretchable strain sensors: Approach and preliminary results,” under review in IEEE Robotics and Automation Letters, submitted Dec. 2024.	paper-1	paper_url

## Import pandas

We are using the very handy pandas library for dataframes.

In [31]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [32]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-10-29,Data-driven Shape Sensing of Continuum Dextero...,IEEE Sensor Conference 2023,NaN,"Q. Li, W. Wang, J. Liu, A. Jain and M. Armand,...",paper-1,https://ieeexplore.ieee.org/abstract/document/...
1,2024-12-19,Shape Sensing of Continuum Robots with Additiv...,IEEE Robotics and Automation Letters,excerpt,"D. C. Moyer, W. Wang, L. S. Karschner, L. Fich...",paper-1,paper_url


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [33]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [34]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [35]:
!ls ../_publications/

   2024-12-19-paper-1.md 2023-10-29-paper-1.md    2024-12-19-paper-1.md


In [36]:
!cat ../_publications/2023-10-29-paper-1.md

---
title: "Data-driven Shape Sensing of Continuum Dexterous Manipulators Using Embedded Capacitive Sensor"
collection: publications
permalink: /publication/2023-10-29-paper-1
date: 2023-10-29
venue: 'IEEE Sensor Conference 2023'
paperurl: 'https://ieeexplore.ieee.org/abstract/document/10324929'
citation: 'Q. Li, W. Wang, J. Liu, A. Jain and M. Armand, &quot;Data-driven Shape Sensing of Continuum Dexterous Manipulators Using Embedded Capacitive Sensor,&quot; 2023 IEEE SENSORS, Vienna, Austria, 2023, pp. 1-4, doi: 10.1109/SENSORS56945.2023.10324929.'
---
[Download paper here](https://ieeexplore.ieee.org/abstract/document/10324929)

Recommended citation: Q. Li, W. Wang, J. Liu, A. Jain and M. Armand, "Data-driven Shape Sensing of Continuum Dexterous Manipulators Using Embedded Capacitive Sensor," 2023 IEEE SENSORS, Vienna, Austria, 2023, pp. 1-4, doi: 10.1109/SENSORS56945.2023.10324929.